In [137]:
import json
import weathercom
import requests
from bs4 import BeautifulSoup

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [142]:

import bitlyshortener
tokens_pool = ['533974390e18ead340d15f7845ab032349f16e10','d30faf515a949e8347e9158a22bf490d3f334cf5']  # Use your own.
shortener = bitlyshortener.Shortener(tokens=tokens_pool, max_cache_size=256)

In [143]:
def save_to_gs():
        # --------------authorize google sheet---------------------
    scopes = ["https://spreadsheets.google.com/feeds"]
    credentials = ServiceAccountCredentials.from_json_keyfile_name("credentials_gs.json", scopes)
    gc = gspread.authorize(credentials)
#     sh = gc.open_by_key("1590Pln7GfV3vvcvvsRNLbMHJzM78Bmtnh9mvTpXBVi4")#juhao
#     sh = gc.open_by_key("1LMoh8cbY2-KdP40Z98DYGWs6r5Dd0aDJWSgxo2f6AII")#english word
    sh = gc.open_by_key("1VqJPzS8pA89MmzdqkYMyrROxQlNfdrtLH-XV2vNU-CU")#weather
    Sheet1 = sh.worksheet("weathercom")
    return Sheet1

In [147]:
def get_url(area):
    weatherDetails = weathercom.getCityWeatherDetails(area)
    res = json.loads(weatherDetails)
    citycode = res['weather.comCityCode']
    url = 'https://weather.com/zh-TW/weather/today/l/' + citycode
    print(url)
    # result = requests.get('https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization=CWB-5B414EA1-34DB-4168-8B56-7EBFE0BBACCF')
    result = requests.get(url)
    soup = BeautifulSoup(result.text,'html.parser')
    long_urls = [url]
    url_short = shortener.shorten_urls(long_urls)
    print(url_short)
    return soup
    # print(soup.prettify())

# search title

In [145]:
def filter_data(soup):
    data = []
    data_1 = []
    data.append(soup.select("title")[0].get_text().split('—')[0]) #title
    data.append(soup.select(".SunriseSunset--dateValue--2nwgx")[0].get_text())#sun rise
    data.append(soup.select(".SunriseSunset--dateValue--2nwgx")[1].get_text())#sunset
    data.append(soup.select('[data-testid="wxPhrase"]')[0].get_text()) #current status
    data.append(soup.select('[data-testid="UVIndexValue"]')[0].get_text()) #紫外綫指數
    data.append(soup.select('[data-testid="VisibilityValue"]')[0].get_text()) #能見度
    data.append(soup.select('[data-testid="PercentageValue"]')[0].get_text()) #濕度
    data.append(soup.select('[data-testid="TemperatureValue"]')[0].get_text()) #露點
    data.append(soup.select('[data-testid="wxData"]')[1].get_text()[14:]) #風向
    data.append(soup.select('[data-testid="wxData"]')[-1].get_text()) #moon
    data.append(soup.select('[data-testid="Description"]')[0].get_text()) #flower
    data.append(soup.select('[data-testid="DonutChartValue"]')[0].get_text()) #AQI
    data.append(soup.select('[data-testid="AirQualityCategory"]')[0].get_text()) #AQI level

    for i in soup.select(".Column--column--2bRa6"):
        data.append(i.select(".Ellipsis--ellipsis--lfjoB")[0].get_text())#time
        data.append(i.select('[data-testid="TemperatureValue"]')[0].get_text()) #temp
        try:
            data.append(i.select('[data-testid="TemperatureValue"]')[1].get_text()) #temp2
        except:
            pass
        data.append(i.select('title')[0].get_text()) #status
        data.append(i.select(".Column--precip--2H5Iw")[0].get_text()[8:]) #rain
        data.append('')

    sheet1 = save_to_gs()
    data_all=[data]
    sheet1.update('A1', data_all)
    return data_all

In [148]:
def main(area):
    soup = get_url(area)
    data_all = filter_data(soup)
    for i in data_all[0]:
        print(i)
    
main('san jose')


https://weather.com/zh-TW/weather/today/l/2e42c7eb8925daeec6059f12994f1a8c0cb3c49d5948965349e3c00f290dca4f
['https://j.mp/2RlDlmz']
聖荷西, CA, 美國天氣預報和天氣狀況
05:59
20:08
多雲
1 (最大值 10)
無限制的
83%
11°
14 km/h
眉月
您所在地區的青草花粉含量高
32
良好
午前
15°
Partly Cloudy
1%

午後
23°
Mostly Sunny


傍晚
14°
Partly Cloudy Night
8%

徹夜
11°
Cloudy
16%

現在
11°
Cloudy
1%

09:00
13°
Partly Cloudy
1%

10:00
15°
Partly Cloudy
1%

11:00
18°
Partly Cloudy


12:00
21°
Partly Cloudy


今天
25°
10°
Partly Cloudy
2%

週六 15
21°
11°
Partly Cloudy
14%

週日 16
23°
11°
Partly Cloudy
7%

週一 17
23°
10°
Partly Cloudy
7%

週二 18
23°
10°
Mostly Sunny
6%



# 获得所有href

In [83]:
soup.select('[data-testid="wxData"]')[-1].get_text() #moon

'殘月'

In [84]:
soup.select(".CurrentConditions--phraseValue--2xXSr")[0].get_text()

'大部多雲'

In [90]:
soup.select('[data-testid="Description"]')[0].get_text() #flower

'您所在地區的青草花粉含量低'

In [90]:
soup.select('[data-testid="Description"]')[0].get_text() #flower

'您所在地區的青草花粉含量低'